## Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import eurostat
import os

In [2]:
from functools import reduce
from eurostat import get_data_df

## Datasets

### Utils

In [130]:
COUNTRY = "SE"
OUTPUT_DIR = "data_eurostat"
os.makedirs(OUTPUT_DIR, exist_ok=True)
DFS = {}
ANNUAL_COLS = []

In [131]:
def get_dataset(indicator_name, local_path):

    # Lookup the dataset inside the data_eurostat folder
    if os.path.exists(local_path):
        print(f"📂 Found locally: {local_path}")
        df = pd.read_csv(local_path)
        return df
    
    # Download if no avilable csv was found
    try:
        print(f"⬇️ Downloading from Eurostat: {indicator_name}")
        df = eurostat.get_data_df(indicator_name)

        if "geo\\TIME_PERIOD" in df.columns:
            df = df.rename(columns={"geo\\TIME_PERIOD":"geo"})

        df.to_csv(local_path, index=False)
        print(f"✅ Saved locally: {local_path}")
    except Exception as e:
        print(f"❌ Failed to download {indicator_name}: {e}")
        return None

    return df

In [132]:
def col_to_timestamp(df, ds_name):

    freq = list(df['freq'].unique())[0]


    try:
        if freq.upper() == 'Q':
            df['TIME_PERIOD'] = pd.PeriodIndex(df['TIME_PERIOD'], freq='Q').to_timestamp()
        elif freq.upper() == 'M':
            df['TIME_PERIOD'] = pd.PeriodIndex(df['TIME_PERIOD'], freq='M').to_timestamp()
        else:
            ANNUAL_COLS.append(ds_name)
            df['TIME_PERIOD'] = pd.to_datetime(df['TIME_PERIOD'])
    
    except Exception as e:
        print(f"Could not parse TIME_PERIOD: {e}")

    return df

### GDP per Capita (Nominal)

In [133]:
indicator_name = "namq_10_gdp"
ds_name = "gdp_per_capita"
filename = f"{ds_name}.csv"
local_path = os.path.join(OUTPUT_DIR, filename)

In [134]:
df = get_dataset(indicator_name=indicator_name, local_path=local_path)

# df.head()

📂 Found locally: data_eurostat\gdp_per_capita.csv


In [135]:
df_clean = df.copy()

df_clean = df_clean[
    (df_clean['geo'] == COUNTRY) & 
    (df_clean['na_item'] == 'B1G') & 
    (df_clean['unit'] == 'CLV05_MEUR') &
    (df_clean['s_adj'] == 'SCA')]

non_null_columns = df_clean.columns[~(df_clean.isnull().sum() > 0)]
df_clean = df_clean[non_null_columns]

id_vars = ['freq', 'geo', 'na_item', 'unit', 's_adj']
value_vars = [col for col in df_clean.columns if col not in id_vars]

df_clean = df_clean.melt(
    id_vars=id_vars,
    value_vars=value_vars,
    var_name='TIME_PERIOD',
    value_name='VALUE'
)

df_clean = df_clean.rename(columns={"VALUE": f"{ds_name}"})
df_clean = col_to_timestamp(df_clean, ds_name)
df_clean = df_clean[['TIME_PERIOD', f'{ds_name}']]

DFS[f"df_{ds_name}"] = df_clean

df_clean.head()

,TIME_PERIOD,gdp_per_capita
0,1993-01-01,46206.9
1,1993-04-01,46621.8
2,1993-07-01,46951.4
3,1993-10-01,47068.7
4,1994-01-01,47692.1


### Unemployment Rate

In [136]:
indicator_name = "tps00203"
ds_name = "unemployment_rate"
filename = f"{ds_name}.csv"
local_path = os.path.join(OUTPUT_DIR, filename)

In [137]:
df = get_dataset(indicator_name=indicator_name, local_path=local_path)

# df.head()

📂 Found locally: data_eurostat\unemployment_rate.csv


In [138]:
df_clean = df.copy()

df_clean = df_clean[
    (df_clean['geo'] == COUNTRY) & 
    (df_clean['unit'] == 'PC_POP') # of total population
]

non_null_columns = df_clean.columns[~(df_clean.isnull().sum() > 0)]
df_clean = df_clean[non_null_columns]

id_vars = ['freq', 'geo', 'sex', 'unit', 'age']
value_vars = [col for col in df_clean.columns if col not in id_vars]

df_clean = df_clean.melt(
    id_vars=id_vars,
    value_vars=value_vars,
    var_name='TIME_PERIOD',
    value_name='VALUE'
)

df_clean = df_clean.rename(columns={"VALUE": f"{ds_name}"})
df_clean = col_to_timestamp(df_clean, ds_name)
df_clean = df_clean[['TIME_PERIOD', f'{ds_name}']]

DFS[f"df_{ds_name}"] = df_clean

df_clean.head(25)

,TIME_PERIOD,unemployment_rate
0,2013-01-01,5.8
1,2014-01-01,5.8
2,2015-01-01,5.4
3,2016-01-01,5.1
4,2017-01-01,4.9
5,2018-01-01,4.7
6,2019-01-01,5.0
7,2020-01-01,6.2
8,2021-01-01,6.6
9,2022-01-01,5.6


### Inflation Rate (HICP)

In [139]:
indicator_name = "tec00118"
ds_name = "inflation_rate"
filename = f"{ds_name}.csv"
local_path = os.path.join(OUTPUT_DIR, filename)

In [140]:
df = get_dataset(indicator_name=indicator_name, local_path=local_path)

# df.head()

📂 Found locally: data_eurostat\inflation_rate.csv


In [141]:
df_clean = df.copy()

df_clean = df_clean[
    (df_clean['geo'] == COUNTRY)
]

non_null_columns = df_clean.columns[~(df_clean.isnull().sum() > 0)]
df_clean = df_clean[non_null_columns]

id_vars = ['freq', 'unit', 'coicop', 'geo']
value_vars = [col for col in df_clean.columns if col not in id_vars]

df_clean = df_clean.melt(
    id_vars=id_vars,
    value_vars=value_vars,
    var_name='TIME_PERIOD',
    value_name='VALUE'
)

df_clean = df_clean.rename(columns={"VALUE": f"{ds_name}"})
df_clean = col_to_timestamp(df_clean, ds_name)
df_clean = df_clean[['TIME_PERIOD', f'{ds_name}']]

DFS[f"df_{ds_name}"] = df_clean

df_clean.head()

,TIME_PERIOD,inflation_rate
0,2013-01-01,0.4
1,2014-01-01,0.2
2,2015-01-01,0.7
3,2016-01-01,1.1
4,2017-01-01,1.9


### Education Attainmanet (25-64)

In [142]:
indicator_name = "tgs00109"
ds_name = "education_attainment"
filename = f"{ds_name}.csv"
local_path = os.path.join(OUTPUT_DIR, filename)

In [143]:
df = get_dataset(indicator_name=indicator_name, local_path=local_path)

# df.head()

📂 Found locally: data_eurostat\education_attainment.csv


In [144]:
df_clean = df.copy()

sweden_geos = ['SE11', 'SE12', 'SE21', 'SE22', 'SE23', 'SE31', 'SE32', 'SE33']

df_clean = (df_clean[df_clean['geo'].isin(sweden_geos)])

df_clean = df_clean[
    (df_clean['sex'] == 'T')
]

non_null_columns = df_clean.columns[~(df_clean.isnull().sum() > 0)]
df_clean = df_clean[non_null_columns]

id_vars = ['freq', 'unit', 'isced11', 'age', 'sex', 'geo']
value_vars = [col for col in df_clean.columns if col not in id_vars]

df_clean = df_clean.melt(
    id_vars=id_vars,
    value_vars=value_vars,
    var_name='TIME_PERIOD',
    value_name='VALUE'
)

df_clean = df_clean.rename(columns={"VALUE": f"{ds_name}"})

df_clean = (
    df_clean.groupby(['TIME_PERIOD', 'freq', 'sex', 'age', 'isced11'])
             [f'{ds_name}']
             .mean()
             .reset_index()
)

df_clean['geo'] = 'SE'
df_clean = col_to_timestamp(df_clean, ds_name)
df_clean = df_clean[['TIME_PERIOD', f'{ds_name}']]

DFS[f"df_{ds_name}"] = df_clean

df_clean.head(25)

,TIME_PERIOD,education_attainment
0,2013-01-01,34.7125
1,2014-01-01,36.1875
2,2015-01-01,37.3000
3,2016-01-01,38.5500
4,2017-01-01,39.2750
5,2018-01-01,40.7375
6,2019-01-01,41.3875
7,2020-01-01,41.8500
8,2021-01-01,43.7375
9,2022-01-01,45.7625


## Merge Datasets

In [165]:
merged_df = None

for name, df_clean in DFS.items():

    if merged_df is None:
        merged_df = df_clean
        continue

    merged_df = pd.merge(merged_df, df_clean, on="TIME_PERIOD", how="outer")

for col in ANNUAL_COLS:
    merged_df = merged_df.ffill()

merged_df = merged_df.dropna()

merged_df = merged_df.sort_values("TIME_PERIOD").reset_index(drop=True)
merged_path = os.path.join(OUTPUT_DIR, "merged_eurostat.csv")
merged_df.to_csv(merged_path, index=False)

merged_df.head(50)

,TIME_PERIOD,gdp_per_capita,unemployment_rate,inflation_rate,education_attainment
0,2013-01-01,78267.7,5.8,0.4,34.7125
1,2013-04-01,78534.3,5.8,0.4,34.7125
2,2013-07-01,79264.6,5.8,0.4,34.7125
3,2013-10-01,80063.5,5.8,0.4,34.7125
4,2014-01-01,80354.8,5.8,0.2,36.1875
5,2014-04-01,80688.9,5.8,0.2,36.1875
6,2014-07-01,81188.2,5.8,0.2,36.1875
7,2014-10-01,81835.5,5.8,0.2,36.1875
8,2015-01-01,82638.0,5.4,0.7,37.3000
9,2015-04-01,84566.1,5.4,0.7,37.3000
